In [17]:
import torch
import os
def find_latest_state_dict(dir: str) -> str:
    latest_state_dict_path = sorted(
        [
            item
            for item in os.listdir(dir)
            if len(item.split("state_dict_epoch")) == 2
        ]
    )[-1]
    return latest_state_dict_path

model_state_dir = "/home/tim/Development/OCELFeatureExtractionExperiments/models/AdamsGCN_20230321_12h34m"
latest_state_dict = find_latest_state_dict(model_state_dir)
best_state_dict = torch.load(os.path.join(model_state_dir,latest_state_dict))


OrderedDict([('gconv1.bias',
              tensor([-1.1441, -1.0989, -0.8061, -1.5397, -1.6912, -1.3571, -1.2048, -1.2405,
                      -1.6152, -1.7674, -1.1258, -0.5485, -1.6483, -1.4768, -1.3514, -2.3590,
                      -1.5854, -2.9206, -1.0138, -0.4594, -2.3181, -1.5687, -1.5297, -0.5714],
                     device='cuda:0')),
             ('gconv1.lin.weight',
              tensor([[ 2.6292e-01, -3.4959e-01, -9.9668e-02, -4.4541e-01, -3.2539e-01,
                        1.9897e+00, -1.3631e+00,  2.1615e+00, -1.4728e+00, -5.2467e-01,
                       -1.2431e-01, -6.2876e-01, -3.2922e-02, -1.0133e-01,  2.6550e-02,
                        6.0127e-02,  8.7439e-01,  5.0503e-01,  4.8757e-01,  4.7201e-01,
                        1.5689e-01, -4.3916e-02,  2.7353e-01, -2.9089e-02],
                      [-6.5871e-01,  3.2123e-01, -6.8691e-01,  2.6495e-01,  6.3250e-02,
                        1.7147e-01,  2.0004e+00, -1.8986e-01,  4.9376e-01,  1.8439e-01,
         

In [28]:
list(best_state_dict.keys())[0]

'gconv1.bias'

: 

In [8]:
# SELF REMINDER. Copy the 'ocpa' directory to the forked one from github, so that I can push updates to github.

# Python native
import os
import random
import pickle
from statistics import median as median
from tqdm import tqdm
from datetime import datetime
from typing import Callable
from sys import platform
if platform == "linux" or platform == "linux2":
    os.chdir("/home/tim/Development/OCELFeatureExtractionExperiments/")
elif platform == "win32":
    os.chdir("c:\\Users\\Tim\\Development\\OCELFeatureExtractionExperiments")

# Object centric process mining
from ocpa.algo.predictive_monitoring.obj import Feature_Storage as FeatureStorage
import ocpa.algo.predictive_monitoring.factory as feature_factory

# Data handling
import numpy as np

# # Simple machine learning models, procedure tools, and evaluation metrics
# from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# PyG
import torch
from model import AdamsGCN
from torch_geometric.loader import DataLoader

# PyTorch TensorBoard support
import torch.utils.tensorboard
from torch.utils.tensorboard.writer import SummaryWriter

from replicating.ocpa_PyG_integration.EventSubGraphDataset import EventSubGraphDataset


# Global variables
from replicating.experiment_config import (
    BATCH_SIZE,
    EPOCHS,
    FEATURE_STORAGE_FILE,
    RANDOM_SEED,
    STORAGE_PATH,
    SUBGRAPH_SIZE,
    TARGET_LABEL,
)


In [9]:
# Adams' settings
RANDOM_SEED = 3
SUBGRAPH_SIZE = 4
EPOCHS = 30
BATCH_SIZE = 64

# Initializing random seeds for maximizing reproducibility
torch.manual_seed(RANDOM_SEED)
np.random.seed(RANDOM_SEED)
random.seed(RANDOM_SEED)
generator = torch.Generator().manual_seed(RANDOM_SEED)
# torch.use_deterministic_algorithms(True) # incompatible with GCN


def seed_worker(worker_id) -> None:
    # worker_seed = torch.initial_seed() % RANDOM_SEED
    worker_seed = RANDOM_SEED
    np.random.seed(worker_seed)
    random.seed(worker_seed)

In [10]:
# If FEATURE_STORAGE_FILE not cached, generate it
if not os.path.exists(f"{STORAGE_PATH}/raw/{FEATURE_STORAGE_FILE}"):
    if not os.path.exists(f"{STORAGE_PATH}/raw/BPI17-feature_storage-[C2,D1,P2,P3,O3].fs"):
        from importing_ocel import feature_storage
    else:
        with open(
            f"{STORAGE_PATH}/raw/BPI17-feature_storage-[C2,D1,P2,P3,O3].fs", "rb"
        ) as file:
            feature_storage: FeatureStorage = pickle.load(file)
    
    # Adams didn't give this split a random seed, 
    # thus we can split the validation set in this arbitrary manner
    feature_storage.extract_normalized_train_test_split(
        test_size=0.3,
        validation_size=0.7*0.2, 
        scaler=StandardScaler,
        scaling_exempt_features=[],
        state=RANDOM_SEED,
    )

    with open(
        f"{STORAGE_PATH}/raw/{FEATURE_STORAGE_FILE}",
        "wb",
    ) as file:
        pickle.dump(feature_storage, file)

ds_train = EventSubGraphDataset(
    train=True,
    root=STORAGE_PATH,
    filename=FEATURE_STORAGE_FILE,
    label_key=TARGET_LABEL,
    size_subgraph_samples=SUBGRAPH_SIZE,
    verbosity=51,
)
ds_val = EventSubGraphDataset(
    validation=True,
    root=STORAGE_PATH,
    filename=FEATURE_STORAGE_FILE,
    label_key=TARGET_LABEL,
    size_subgraph_samples=SUBGRAPH_SIZE,
    verbosity=51,
)
ds_test = EventSubGraphDataset(
    test=True,
    root=STORAGE_PATH,
    filename=FEATURE_STORAGE_FILE,
    label_key=TARGET_LABEL,
    size_subgraph_samples=SUBGRAPH_SIZE,
    verbosity=51,
)

No EventSubGraphDataset found with this configuration in 'data/ocpa-processed/processed'. Proceeding to processing...


Processing...
17645it [01:32, 190.80it/s]
Done!


No EventSubGraphDataset found with this configuration in 'data/ocpa-processed/processed'. Proceeding to processing...


Processing...
4411it [00:21, 206.14it/s]
Done!


No EventSubGraphDataset found with this configuration in 'data/ocpa-processed/processed'. Proceeding to processing...


Processing...
9453it [00:47, 198.15it/s]
Done!


In [6]:
print("Train set")
print(ds_train.get_summary())
print()

# Val and test look the same (most likely)
print("Validation set")
print(ds_val.get_summary())
print()

print("Test set")
print(ds_test.get_summary())
print()

Train set


100%|██████████| 34854/34854 [00:10<00:00, 3265.12it/s]


EventSubGraphDataset (#graphs=34854):
+------------+----------+----------+
|            |   #nodes |   #edges |
|------------+----------+----------|
| mean       |       12 |     12.2 |
| std        |        0 |      1.2 |
| min        |       12 |      6   |
| quantile25 |       12 |     12   |
| median     |       12 |     12   |
| quantile75 |       12 |     13   |
| max        |       12 |     19   |
+------------+----------+----------+

Validation set
EventSubGraphDataset (#graphs=8744):
+------------+----------+----------+
|            |   #nodes |   #edges |
|------------+----------+----------|
| mean       |       12 |     12.3 |
| std        |        0 |      1.2 |
| min        |       12 |      9   |
| quantile25 |       12 |     12   |
| median     |       12 |     12   |
| quantile75 |       12 |     13   |
| max        |       12 |     19   |
+------------+----------+----------+

Test set


100%|██████████| 18072/18072 [00:04<00:00, 3937.56it/s]

EventSubGraphDataset (#graphs=18072):
+------------+----------+----------+
|            |   #nodes |   #edges |
|------------+----------+----------|
| mean       |       12 |     12.2 |
| std        |        0 |      1.2 |
| min        |       12 |      8   |
| quantile25 |       12 |     12   |
| median     |       12 |     12   |
| quantile75 |       12 |     13   |
| max        |       12 |     19   |
+------------+----------+----------+



In [4]:
def count_parameters(model) -> int:
    return sum(p.numel() for p in model.parameters() if p.requires_grad)


# Initialize model
model = AdamsGCN(
    num_node_features=ds_train.num_node_features,
    hyperparams={
        "num_hidden_features": ds_train.num_node_features,
        "size_subgraph_samples": SUBGRAPH_SIZE,
    },
)
print(model)
print(f"Number of parameters: {count_parameters(model)}")


# Use GPU
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(f"Device: {device}")
model = model.to(device)
# data = ds_train.to(device)

# Initialize Optimizer
learning_rate = 0.01
# decay = 5e-4
optimizer = torch.optim.Adam(
    model.parameters(),
    lr=learning_rate,
    # weight_decay=decay
)

mse = torch.nn.MSELoss()
mae = torch.nn.L1Loss()
loss_fn = mae

train_loader = DataLoader(
    ds_train,
    batch_size=BATCH_SIZE,
    shuffle=False,
    pin_memory=True,
    num_workers = 4,
    worker_init_fn=seed_worker,
    generator=generator,
)
val_loader = DataLoader(
    ds_val,
    batch_size=BATCH_SIZE,
    shuffle=False,
    pin_memory=True,
    num_workers = 4,
    worker_init_fn=seed_worker,
    generator=generator,
)
test_loader = DataLoader(
    ds_test,
    batch_size=128,
    shuffle=False,
    pin_memory=True,
    num_workers = 4,
    worker_init_fn=seed_worker,
    generator=generator,
)


def train_one_epoch(
    epoch_index: int, model, train_loader, optimizer, loss_fn, tb_writer
) -> float:
    # Enumerate over the data
    running_loss = 0.0
    last_loss = 0
    for i, batch in enumerate(tqdm(train_loader)):
        # Use GPU
        batch.to(device)
        # Every data instance is an input + label pair
        inputs, adjacency_matrix, labels = (
            batch.x.float(),  # k times the batch_size, where k is the subgraph size
            batch.edge_index,
            batch.y.float(),
        )
        # Reset gradients (set_to_none is faster than to zero)
        optimizer.zero_grad(set_to_none=True)
        # Passing the node features and the connection info
        outputs = model(inputs, adjacency_matrix)
        # Compute loss and gradients
        loss = loss_fn(torch.squeeze(outputs), labels)
        loss.backward()
        # Adjust learnable weights
        optimizer.step()
        # Gather data and report
        running_loss += loss.item()
        if i % 1000 == 999:
            last_loss = running_loss / 1000  # loss per batch
            print(f"  batch {i + 1} loss: {last_loss}")
            tb_x = epoch_index * len(train_loader) + i + 1
            tb_writer.add_scalar("Loss/train", last_loss, tb_x)
            running_loss = 0.0

    return last_loss


def run_training(
    num_epochs, model, train_loader, validation_loader, optimizer, loss_fn, timestamp
):
    model_path = f"models/{model.get_class_name()}_{timestamp}"
    if not os.path.exists(model_path):
        os.makedirs(model_path)
    writer = SummaryWriter(f"{model_path}/run")
    best_vloss = 1_000_000_000_000_000.0

    for epoch in range(num_epochs):
        print(f"EPOCH {epoch + 1}:")

        # Make sure gradient tracking is on, and do a pass over the data
        model.train(True)
        avg_loss = train_one_epoch(
            epoch, model, train_loader, optimizer, loss_fn, writer
        )

        # We don't need gradients on to do reporting
        model.train(False)

        running_vloss = 0.0
        for i, vdata in enumerate(validation_loader):
            vdata.to(device)
            vinputs, vadjacency_matrix, vlabels = (
                vdata.x.float(),
                vdata.edge_index,
                vdata.y.float(),
            )
            voutputs = model(vinputs, vadjacency_matrix)
            vloss = loss_fn(voutputs, vlabels)
            running_vloss += vloss

        avg_vloss = running_vloss / (i + 1)
        print(f"LOSS train {avg_loss} valid {avg_vloss}")

        # Log the running loss averaged per batch
        # for both training and validation
        writer.add_scalars(
            "Training vs. Validation Loss",
            {"Training": avg_loss, "Validation": avg_vloss},
            epoch + 1,
        )
        writer.flush()

        # Track best performance, and save the model's state
        if avg_vloss < best_vloss:
            best_vloss = avg_vloss
            torch.save(model.state_dict(), f"{model_path}/state_dict_epoch{epoch}.pt")

def evaluate_model(
    model,
    dataloader,
    metric: Callable[[torch.Tensor, torch.Tensor], torch.Tensor],
    device: torch.device = torch.device("cpu"),
    verbose: bool = False,
) -> torch.Tensor:
    with torch.no_grad():

        def _eval_batch(batch, model):
            batch_inputs, batch_adjacency_matrix, batch_labels = (
                batch.x.float(),
                batch.edge_index,
                batch.y.float(),
            )
            return model(batch_inputs, batch_adjacency_matrix), batch_labels

        model.eval()
        model.train(False)
        model.to(device)
        y_preds = torch.tensor([]).to(device)
        y_true = torch.tensor([]).to(device)
        for batch in tqdm(dataloader, disable=not (verbose)):
            batch.to(device)
            batch_y_preds, batch_y_true = _eval_batch(batch, model)
            y_preds = torch.cat((y_preds, batch_y_preds))
            y_true = torch.cat((y_true, batch_y_true))
        y_preds = torch.squeeze(y_preds)
    return metric(y_preds.to(device), y_true.to(device))

IndexError: range object index out of range

In [28]:

# Initializing in a separate cell so we can easily add more epochs to the same run
timestamp = datetime.now().strftime("%Y%m%d_%Hh%Mm")

run_training(
    num_epochs=EPOCHS,
    model=model,
    train_loader=train_loader,
    validation_loader=val_loader,
    optimizer=optimizer,
    loss_fn=loss_fn,
    timestamp=timestamp,
)


EPOCH 1:


 39%|███▊      | 1012/2626 [00:13<00:18, 85.94it/s]

  batch 1000 loss: 0.4966896296143532


 77%|███████▋  | 2010/2626 [00:25<00:06, 91.09it/s]

  batch 2000 loss: 0.4746956475973129


...

  batch 1000 loss: 0.4377897444963455


 77%|███████▋  | 2009/2626 [00:22<00:06, 89.08it/s] 

  batch 2000 loss: 0.4464273728579283


100%|██████████| 2626/2626 [00:29<00:00, 89.94it/s] 


LOSS train 0.4464273728579283 valid 0.8942859768867493


In [29]:
model_path0 = "/home/tim/Development/OCELFeatureExtractionExperiments/models/GCN_20230315_17h07m/state_dict_epoch26.pt"
model_path = "/home/tim/Development/OCELFeatureExtractionExperiments/models/GCN_20230315_23h22m/state_dict_epoch6.pt"
adams_model_path = "/home/tim/Development/OCELFeatureExtractionExperiments/models/AdamsGCN_20230321_12h34m/state_dict_epoch3.pt"

model.load_state_dict(torch.load(adams_model_path))

<All keys matched successfully>

In [36]:
normed_train_mae = evaluate_model(
    model=model,
    dataloader=train_loader,
    metric=mae,
    device=device,
    verbose=True,
)
normed_train_mae

100%|██████████| 2626/2626 [00:25<00:00, 103.82it/s]


tensor(0.4410)

In [37]:
normed_val_mae = evaluate_model(
    model=model,
    dataloader=val_loader,
    metric=mae,
    device=device,
    verbose=True,
)
normed_val_mae

100%|██████████| 657/657 [00:09<00:00, 67.98it/s] 


tensor(0.4512)

In [35]:
normed_test_mae = evaluate_model(
    model=model,
    dataloader=test_loader,
    metric=mae,
    device=device,
    verbose=True,
)
normed_test_mae

100%|██████████| 699/699 [00:12<00:00, 54.42it/s]


tensor(0.4354)

In [38]:
with open(
    f"{STORAGE_PATH}/raw/{FEATURE_STORAGE_FILE}",
    "rb",
) as file:
    fs: FeatureStorage = pickle.load(file)


In [39]:
train_mae = fs.scaler.inverse_transform([normed_train_mae]*25)[-2]
val_mae = fs.scaler.inverse_transform([normed_val_mae]*25)[-2]
test_mae = fs.scaler.inverse_transform([normed_test_mae]*25)[-2]

print("Mean absolute error denormalized:")
print('Train MAE: ', train_mae)
print('Val MAE:   ', val_mae)
print('Test MAE:  ', test_mae)

Mean absolute error denormalized:
Train MAE:  1717909.0
Val MAE:    1730090.6
Test MAE:   1711158.8
